<a href="https://colab.research.google.com/github/OjochenemiB/Supervised_Learning/blob/main/Brain_Tumour_Prediction(3types)_Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import layers,Input,Model
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
import os
from pathlib import Path
import cv2
from PIL import Image # pillow library
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import RandomOverSampler, SMOTE, SVMSMOTE
from sklearn.utils import resample
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Import Image_dataset

In [2]:

# Get the current working directory of the script
current_directory = Path.cwd()
dataset_directory = current_directory / "brain_tumor_dataset_3"

# Ensure the folder exists
if not dataset_directory.exists():
    raise FileNotFoundError(f"The folder {dataset_directory} does not exist.")

#CREATE A DICTIONARY TO STORE ALL THE CLASSES OF BRAIN TUMOUR
brain_tumour_dic = {'No_tumor' : list(dataset_directory.glob('notumor/*')),
                    'glioma' : list(dataset_directory.glob('glioma/*')),
                    'meningioma' : list(dataset_directory.glob('meningioma/*')),
                    'pituitary' : list(dataset_directory.glob('pituitary/*'))
                   }
print("Total Image is ", len(brain_tumour_dic['No_tumor'])+ len(brain_tumour_dic['glioma'])+ len(brain_tumour_dic['meningioma'])+ len(brain_tumour_dic['pituitary']),"\n")

print(brain_tumour_dic.keys())

len(brain_tumour_dic['No_tumor']), len(brain_tumour_dic['glioma']), len(brain_tumour_dic['meningioma']), len(brain_tumour_dic['pituitary'])

FileNotFoundError: The folder /content/brain_tumor_dataset_3 does not exist.

### View Dataset

In [ ]:
Image.open(str(brain_tumour_dic['No_tumor'][3]))

In [ ]:
Image.open(brain_tumour_dic['pituitary'][5])

In [ ]:
brain_tumour_label = {'No_tumor':0,
                     'glioma':1,
                      'meningioma':2,
                      'pituitary':3
                    }

In [ ]:
img_conv = cv2.imread(brain_tumour_dic ['pituitary'][4])
img_conv.shape

### Resize all Images

In [ ]:
# Initialize lists to hold images and labels
images = []
labels = []
# Load images and labels
for label_name, image_paths in brain_tumour_dic.items():
    label = brain_tumour_label[label_name]
    for img_path in image_paths:
        # Load image
        image = Image.open(img_path).convert('L')  # Convert to grayscale
        image = image.resize((128, 128))  # Resize to desired size
        image = np.array(image) / 255.0  # Normalize to [0, 1]

        images.append(image)
        labels.append(label)

In [ ]:
# convert into a numpy array
resized_image = np.array(images)
image_label = np.array(labels)

# Ensure correct shape: (num_samples, height, width, channels)
resized_image = np.expand_dims(resized_image, axis=-1)  # Add channel dimension

### Split Dataset

In [ ]:
Xtrain,Xtest, Ytrain,Ytest = train_test_split(resized_image, image_label,test_size=0.2,random_state=42)

print("Training data shape:", Xtrain.shape)
print("Validation data shape:", Xtest.shape)
print("Training labels shape:", Ytrain.shape)
print("Validation labels shape:", Ytest.shape)

### One-Hot Encode Labels

In [ ]:
num_classes = 4  # Number of classes

# Function to convert class labels to one-hot encoding
def one_hot_encode_labels(y_data, height, width, num_classes):
    # y_data should have shape (num_samples,)
    # Create an empty array with the desired shape
    y_one_hot = np.zeros((y_data.shape[0], height, width, num_classes), dtype=np.float32)

    for i in range(y_data.shape[0]):
        label = y_data[i]
        y_one_hot[i, :, :, label] = 1

    return y_one_hot

height, width = 128, 128  # Adjust according to your image size
y_train_one_hot = one_hot_encode_labels(Ytrain, height, width, num_classes)
y_val_one_hot = one_hot_encode_labels(Ytest, height, width, num_classes)

print("One-hot encoded training labels shape:", y_train_one_hot.shape)
print("One-hot encoded validation labels shape:", y_val_one_hot.shape)

### Class Balancing

## Image Segmentation

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model

def unet_model(input_size, num_classes=4):
    inputs = Input(input_size)

    # Downsample
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p2)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Upsample
    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c2])
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c1])
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c7)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [ ]:
# Define input size
input_size = (128, 128, 1)
num_classes=4
# Get the U-Net model
model = unet_model(input_size,num_classes)

# Compile the model with categorical_crossentropy loss
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

### Data Augumentation

In [ ]:
# Define data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(Xtrain)

In [ ]:
# Example training
history = model.fit(
    datagen.flow(Xtrain, y_train_one_hot, batch_size=16),
    steps_per_epoch=len(Xtrain) // 16,
    epochs=10,
    validation_data=(Xtest, y_val_one_hot)
)